In [86]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from transformers import pipeline, TextStreamer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain

In [2]:
# CHAT_MODEL_PATH = "/mnt/nfs/zsd_server/models/huggingface/chinese-alpaca-2-7b/"
CHAT_MODEL_PATH = "/mnt/nfs/zsd_server/models/huggingface/llama-3-chinese-8b-instruct-v3/"

In [3]:
chat_model = AutoModelForCausalLM.from_pretrained(
    CHAT_MODEL_PATH,
    # load_in_8bit=True,
    device_map='auto',
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL_PATH)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.64it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [43]:
streamer = TextStreamer(tokenizer)
pipe = pipeline(
    "text-generation",
    model=chat_model,
    tokenizer=tokenizer,
    max_length=4096,
    truncation=True,
    repetition_penalty=1.2,
    do_sample=True,
    temperature=0.6,
    top_p=0.95,
    streamer=streamer,
    return_full_text=False,
)
local_llm = HuggingFacePipeline(pipeline=pipe)

In [111]:
text_contents = '''自2009年创立之初，阿里云就提出“云计算，
让计算成为公共服务”，并坚持通过云的弹性和自服务能力支持企业敏捷创新。
自2016年起，阿里云持续保持中国第一、亚太第一、全球第三的市场地位。
当前阿里云服务超400万客户，包括60%A股上市公司。
'''
fact_template = PromptTemplate(
    input_variables=["text_input"],
    template=("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>你是一个分析师。请从本文中提取关键事实。不要包括意见。给每个事实一个数字，并保持简短的句子。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: {text_input}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
"""),
)

fact_extraction_chain = LLMChain(
    llm=local_llm, prompt=fact_template, output_key="facts")

facts = fact_extraction_chain.run(text_contents)
facts

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>你是一个分析师。请从本文中提取关键事实。不要包括意见。给每个事实一个数字，并保持简短的句子。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: 自2009年创立之初，阿里云就提出“云计算，
让计算成为公共服务”，并坚持通过云的弹性和自服务能力支持企业敏捷创新。
自2016年起，阿里云持续保持中国第一、亚太第一、全球第三的市场地位。
当前阿里云服务超400万客户，包括60%A股上市公司。
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
1. 阿里云在2009年成立时宣称："云计算，让计算成为公共服务"
2. 阿里云始终以其弹性的特点来支撑企业快速迭代与发展。
3. 从2016年开始，阿里云一直是中国第一、亚太地区最大的云服务商，以及全球排名第三的大型云提供者之一。
4. 目前，阿里云已有超过400多万名用户，其中约占到全行业的六成以上，是国内外知名大型机构及互联网巨头共同选择的首选云平台。

注意：这些都是客观的事实，不包含任何主观看法或评价。<|eot_id|>


'1. 阿里云在2009年成立时宣称："云计算，让计算成为公共服务"\n2. 阿里云始终以其弹性的特点来支撑企业快速迭代与发展。\n3. 从2016年开始，阿里云一直是中国第一、亚太地区最大的云服务商，以及全球排名第三的大型云提供者之一。\n4. 目前，阿里云已有超过400多万名用户，其中约占到全行业的六成以上，是国内外知名大型机构及互联网巨头共同选择的首选云平台。\n\n注意：这些都是客观的事实，不包含任何主观看法或评价。'

In [112]:
role_template = PromptTemplate(
    input_variables=["facts"],
    template=("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请根据以下事实清单，用中文为投资者写一个简短的段落。不要遗漏关键信息。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: {facts}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
"""),
)

data_analysis_chain = LLMChain(llm=local_llm, prompt=role_template, output_key="analysis")

data_analysis_facts = data_analysis_chain.run(facts)
data_analysis_facts

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请根据以下事实清单，用中文为投资者写一个简短的段落。不要遗漏关键信息。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: 1. 阿里云在2009年成立时宣称："云计算，让计算成为公共服务"
2. 阿里云始终以其弹性的特点来支撑企业快速迭代与发展。
3. 从2016年开始，阿里云一直是中国第一、亚太地区最大的云服务商，以及全球排名第三的大型云提供者之一。
4. 目前，阿里云已有超过400多万名用户，其中约占到全行业的六成以上，是国内外知名大型机构及互联网巨头共同选择的首选云平台。

注意：这些都是客观的事实，不包含任何主观看法或评价。<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
阿里云自创立以来就秉持着“让计算成为公共服务”的理念，并凭借其独有的弹性优势，为各类企业带来了极高效率和灵活度，以支持他们迅速适应市场变化并持续创新。此后，在近十年的时间内，它不断壮大自身的地位，从2016年起稳居中国第一、大陆第二的云服务领导地位，并跻身于世界三甲之列。在今天看来，阿里云已经拥有了超四百万名客户群体，这其中包括国民级别的大规模运营商、小微企业以及来自不同领域的众多领先品牌。这使得它不仅获得了广泛认可，也逐渐演变成了业界公认的一流云解决方案供给商。无论是在国内还是国际舞台上，都能见证阿里云作为一家真正关注科技革新且致力于推动数字化转型的公司所取得的卓越成绩。因此，对于寻求切实有效帮助实现业务目标和加快技术进步的人来说，阿里云是一个值得信赖且具备潜力的合作伙伴。<|eot_id|>


'阿里云自创立以来就秉持着“让计算成为公共服务”的理念，并凭借其独有的弹性优势，为各类企业带来了极高效率和灵活度，以支持他们迅速适应市场变化并持续创新。此后，在近十年的时间内，它不断壮大自身的地位，从2016年起稳居中国第一、大陆第二的云服务领导地位，并跻身于世界三甲之列。在今天看来，阿里云已经拥有了超四百万名客户群体，这其中包括国民级别的大规模运营商、小微企业以及来自不同领域的众多领先品牌。这使得它不仅获得了广泛认可，也逐渐演变成了业界公认的一流云解决方案供给商。无论是在国内还是国际舞台上，都能见证阿里云作为一家真正关注科技革新且致力于推动数字化转型的公司所取得的卓越成绩。因此，对于寻求切实有效帮助实现业务目标和加快技术进步的人来说，阿里云是一个值得信赖且具备潜力的合作伙伴。'

In [113]:
knowledge_template = PromptTemplate(
    input_variables=["facts"],
    template=("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请用中文，将以下事实列表转换为知识图谱的三元组。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: {facts}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
""")
)

knowledge_chain = LLMChain(llm=local_llm, prompt=knowledge_template, output_key="knowledge")

knowledge_graph = knowledge_chain.run(facts)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请用中文，将以下事实列表转换为知识图谱的三元组。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: 1. 阿里云在2009年成立时宣称："云计算，让计算成为公共服务"
2. 阿里云始终以其弹性的特点来支撑企业快速迭代与发展。
3. 从2016年开始，阿里云一直是中国第一、亚太地区最大的云服务商，以及全球排名第三的大型云提供者之一。
4. 目前，阿里云已有超过400多万名用户，其中约占到全行业的六成以上，是国内外知名大型机构及互联网巨头共同选择的首选云平台。

注意：这些都是客观的事实，不包含任何主观看法或评价。<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
（阿里云，成立时间，2009）
（阿里云，声明内容，云计算让计算成为公共服务）
（阿里云，特点描述，以弹性支持企业快捷迭代和发展）

（阿里云，起始年份，2016）
（阿里云，地理位置限定，中国）
（阿里云，职位定级，一直保持着亚太区最大以及全球排名前列之地位）

（阿里云，用户数量，大于等于400万）
（阿里云，市场占比，高达六成左右）
（阿里云，被认可的地位，为国内外各类知名机构及互联网公司所青睐）<|eot_id|>


In [122]:
# https://python.langchain.com.cn/docs/modules/chains/foundational/sequential_chains

In [114]:
simple_sequential_chain = SimpleSequentialChain(chains=[fact_extraction_chain, data_analysis_chain], verbose=True)
simple_sequential_chain

SimpleSequentialChain(verbose=True, chains=[LLMChain(prompt=PromptTemplate(input_variables=['text_input'], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>你是一个分析师。请从本文中提取关键事实。不要包括意见。给每个事实一个数字，并保持简短的句子。<|eot_id|>\n<|start_header_id|>user<|end_header_id|>: {text_input}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>:\n'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x771674ff3580>), output_key='facts'), LLMChain(prompt=PromptTemplate(input_variables=['facts'], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请根据以下事实清单，用中文为投资者写一个简短的段落。不要遗漏关键信息。<|eot_id|>\n<|start_header_id|>user<|end_header_id|>: {facts}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>:\n'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x771674ff3580>), output_key='analysis')])

In [115]:
simple_sequential_result = simple_sequential_chain.run(text_contents)
simple_sequential_result

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new SimpleSequentialChain chain...
<|begin_of_text|><|start_header_id|>system<|end_header_id|>你是一个分析师。请从本文中提取关键事实。不要包括意见。给每个事实一个数字，并保持简短的句子。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: 自2009年创立之初，阿里云就提出“云计算，
让计算成为公共服务”，并坚持通过云的弹性和自服务能力支持企业敏捷创新。
自2016年起，阿里云持续保持中国第一、亚太第一、全球第三的市场地位。
当前阿里云服务超400万客户，包括60%A股上市公司。
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
1. 阿里云于2009年成立时宣称："云计算，让计算成为公共服务"。
2. 阿里云一直以来都致力于利用云平台的弹性与自助式特点来推动企业进行快速创新。
3. 从2016年开始，阿里云连续占据了中国国内市场第一名、亚洲及太平洋地区第一名以及全球第三名的地位。
4. 目前，阿里云已为超过四百万用户提供服务，其中六成以上是A股上市公司的一部分。<|eot_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1. 阿里云于2009年成立时宣称："云计算，让计算成为公共服务"。
2. 阿里云一直以来都致力于利用云平台的弹性与自助式特点来推动企业进行快速创新。
3. 从2016年开始，阿里云连续占据了中国国内市场第一名、亚洲及太平洋地区第一名以及全球第三名的地位。
4. 目前，阿里云已为超过四百万用户提供服务，其中六成以上是A股上市公司的一部分。
<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请根据以下事实清单，用中文为投资者写一个简短的段落。不要遗漏关键信息。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: 1. 阿里云于2009年成立时宣称："云计算，让计算成为公共服务"。
2. 阿里云一直以来都致力于利用云平台的弹性与自助式特点来推动企业进行快速创新。
3. 从2016年开始，阿里云连续占据了中国国内市场第一名、亚洲及太平洋地区第一名以及全球第三名的地位。
4. 目前，阿里云已为超过四百万用户提供服务，其中六成以上是A股上市公司的一部分。<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
作为一家领先的云计算解决方案供应商，阿里云从其诞生之日起就以「让计算成为公共服务」为宗旨，并始终坚持将云技术应用到各行各业中，以实现更快捷、高效和灵活化的业务运作方式。在过去几年的发展过程中，阿里云凭借着强大的产品线和卓越的服务质量，不断扩大自己的影响范围并稳步提升在国际上的地位，从而跻身世界三甲。这也得益于阿里云不断完善自身的竞争优势，如高性能、大规模存储等方面，以及优质客户体验。此外，目前阿里云已经拥有超400多万个忠诚用户，其中包括大量来自资本市场的知名机构，这些合作伙伴对阿里云所提供的全方位IT基础设施支持给予高度评价，为未来的增长奠定良好的基石。总结来说，阿里云无疑是一个值得信赖且具有潜力的云计算巨头，它不仅能满足企业的需求，还能够帮助他们应对未来挑战，在激烈的行业竞争环境下保持持续成功。<|eot_id|>
作为一家领先的云计算解决方案供应商，阿里云从其诞生之日起就以「让计算成为公共服务」为宗旨，并始终坚持将云技术应用到各行各业中，以实现更快捷、高效和灵活化的业务

'作为一家领先的云计算解决方案供应商，阿里云从其诞生之日起就以「让计算成为公共服务」为宗旨，并始终坚持将云技术应用到各行各业中，以实现更快捷、高效和灵活化的业务运作方式。在过去几年的发展过程中，阿里云凭借着强大的产品线和卓越的服务质量，不断扩大自己的影响范围并稳步提升在国际上的地位，从而跻身世界三甲。这也得益于阿里云不断完善自身的竞争优势，如高性能、大规模存储等方面，以及优质客户体验。此外，目前阿里云已经拥有超400多万个忠诚用户，其中包括大量来自资本市场的知名机构，这些合作伙伴对阿里云所提供的全方位IT基础设施支持给予高度评价，为未来的增长奠定良好的基石。总结来说，阿里云无疑是一个值得信赖且具有潜力的云计算巨头，它不仅能满足企业的需求，还能够帮助他们应对未来挑战，在激烈的行业竞争环境下保持持续成功。'

In [118]:
sequential_chain = SequentialChain(
    chains=[fact_extraction_chain, data_analysis_chain, knowledge_chain],
    input_variables=["text_input"],
    # Here we return multiple variables
    output_variables=["analysis", "knowledge"],
    verbose=True)

In [121]:
sequential_chain_result = sequential_chain({"text_input": text_contents})
sequential_chain_result

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new SequentialChain chain...
<|begin_of_text|><|start_header_id|>system<|end_header_id|>你是一个分析师。请从本文中提取关键事实。不要包括意见。给每个事实一个数字，并保持简短的句子。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: 自2009年创立之初，阿里云就提出“云计算，
让计算成为公共服务”，并坚持通过云的弹性和自服务能力支持企业敏捷创新。
自2016年起，阿里云持续保持中国第一、亚太第一、全球第三的市场地位。
当前阿里云服务超400万客户，包括60%A股上市公司。
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
1. 阿里云于2009年成立时宣称："云计算，让计算成为公共服务"
2. 阿里云在2016年后一直维护着中国第一、亚洲第一以及全世界第三的地位。
3. 目前有超过四百万名用户使用了阿里云，其中六成以上是A股上市公司。<|eot_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请根据以下事实清单，用中文为投资者写一个简短的段落。不要遗漏关键信息。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: 1. 阿里云于2009年成立时宣称："云计算，让计算成为公共服务"
2. 阿里云在2016年后一直维护着中国第一、亚洲第一以及全世界第三的地位。
3. 目前有超过四百万名用户使用了阿里云，其中六成以上是A股上市公司。<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
阿里云自2009年创立以来，一直以推动云计算技术普及和应用而闻名。在其初期，阿里云就提出了一句具有颠覆性的口号：“云计算，让计算成为公共服务”，旨在打破传统IT资源分配方式，为企业提供更加灵活、高效且经济可行的解决方案。

如今，在多年的发展中，阿里云已经取得了显著成绩，并成功地保持住全球顶尖水平之一。这得益于不断完善的产品线与服务体系，以及对客户需求进行深入理解并快速响应的能力。此外，它也积极拓展国际市场，与各大国家合作共建数字基础设施，加速新一代互联网产业的升级转型。

值得注意的是，目前已拥有逾400万个用户群体中的三分之二，即约260余万家企业选择使用阿里云，这其中包括众多知名的大型机构，如许多国内领先的科技巨头、中小微型企业等。而更具代表性的一点就是，有近六成以上的上市公司都将业务迁移到了阿里云平台上来享受高性能、大容量存储、小规模部署等优势，从而实现自身运营成本优化、创新加快步伐等目标。因此，对于那些寻求高质量云计算解决方案的投资者来说，可以考虑关注阿里云这一行业龙头，以期待未来更多的增长机会！<|eot_id|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请用中文，将以下事实列表转换为知识图谱的三元组。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: 1. 阿里云于2009年成立时宣称："云计算，让计算成为公共服务"
2. 阿里云在2016年后一直维护着中国第一、亚洲第一以及全世界第三的地位。
3. 目前有超过四百万名用户使用了阿里云，其中六成以上是A股上市公司。<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
（阿里云，成立时间，2009）
（阿里云，宣言内容，“云计算，让计算成为公共服务”）
（阿里云，地位变化描述，“从2016年起保持中国第一、亚洲第一和全球第 三”的位置）

（阿里云，用户数量，大于400万）
（阿里云，用户比例，一半以上来自于A股上市公司）<|eot_id|>

> Finished chain.


{'text_input': '自2009年创立之初，阿里云就提出“云计算，\n让计算成为公共服务”，并坚持通过云的弹性和自服务能力支持企业敏捷创新。\n自2016年起，阿里云持续保持中国第一、亚太第一、全球第三的市场地位。\n当前阿里云服务超400万客户，包括60%A股上市公司。\n',
 'analysis': '阿里云自2009年创立以来，一直以推动云计算技术普及和应用而闻名。在其初期，阿里云就提出了一句具有颠覆性的口号：“云计算，让计算成为公共服务”，旨在打破传统IT资源分配方式，为企业提供更加灵活、高效且经济可行的解决方案。\n\n如今，在多年的发展中，阿里云已经取得了显著成绩，并成功地保持住全球顶尖水平之一。这得益于不断完善的产品线与服务体系，以及对客户需求进行深入理解并快速响应的能力。此外，它也积极拓展国际市场，与各大国家合作共建数字基础设施，加速新一代互联网产业的升级转型。\n\n值得注意的是，目前已拥有逾400万个用户群体中的三分之二，即约260余万家企业选择使用阿里云，这其中包括众多知名的大型机构，如许多国内领先的科技巨头、中小微型企业等。而更具代表性的一点就是，有近六成以上的上市公司都将业务迁移到了阿里云平台上来享受高性能、大容量存储、小规模部署等优势，从而实现自身运营成本优化、创新加快步伐等目标。因此，对于那些寻求高质量云计算解决方案的投资者来说，可以考虑关注阿里云这一行业龙头，以期待未来更多的增长机会！',
 'knowledge': '（阿里云，成立时间，2009）\n（阿里云，宣言内容，“云计算，让计算成为公共服务”）\n（阿里云，地位变化描述，“从2016年起保持中国第一、亚洲第一和全球第 三”的位置）\n\n（阿里云，用户数量，大于400万）\n（阿里云，用户比例，一半以上来自于A股上市公司）'}